### GERM 
##### Module com ferramentas para construir, simular e analisar GEnome-scale Regulatory and Metabolic (GERM) models


In [2]:
!tree ../src/mewpy/germ -L 1 -I "__*"

../src/mewpy/germ
├── algebra
├── analysis
├── lp
├── models
├── solution
└── variables

6 directories, 0 files


#### GERM/Models
##### Representação de modelos no mewpy. Modelos:
1. Regulatórios
2. Metabólicos
3. Regulatórios + Metabólicos

In [3]:
!tree ../src/mewpy/germ/models -L 1 -I "__*"

../src/mewpy/germ/models
├── cobra_wrapper.py
├── metabolic.py
├── mewpy_metabolic.py
├── model.py
├── reframed_wrapper.py
├── regulatory.py
└── serialization.py

0 directories, 7 files


#### GERM models architecture
<img src="images/germmodels_arch.png" alt="GERM models Architecture" width="600">

##### Para instanciar estes modelos podemos usar a função: 

```python
mewpy.io.read_model()

- Input: 1 ou 2 Readers
- Output: Model (Regulatório | Metabólico | Regulatório + Metabólico)


In [8]:
from mewpy.io import read_model
help(read_model)

Help on function read_model in module mewpy.io:

read_model(*readers: mewpy.io.reader.Reader, warnings: bool = True) -> Union[ForwardRef('Model'), ForwardRef('RegulatoryModel'), ForwardRef('MetabolicModel')]
    Reading a GERM model encoded into one or more file types (e.g. sbml, csv, cobrapy, reframed, json, etc).
    It can return a metabolic, regulatory or metabolic-regulatory model from multiple files.
    
    A reader must be provided for each file type.
    Reading will take place according to the order and settings of the each reader.
    
    All files are closed upon reading or failure
    
    :param readers: Multiple Reader instances that will be used to read multiple file types into a single GERM model
    :param warnings: Whether to launch warnings found during reading
    :return: mewpy metabolic, regulatory or both model



##### Os Readers são objetos utilizados para ler um modelo.

```python
mewpy.io.reader

- Engine: *lista abaixo*
- IO: representação de um modelo (.sbml, .csv, CobraModel, ReframedModel, etc…)


In [11]:
# Exemplo de instancia de um reader
from mewpy.io import Engines
from mewpy.io.reader import Reader
from cobra.io import read_sbml_model

cobra_model = read_sbml_model("./models/e_coli_core.xml")

Reader(Engines.CobraModelEngine, cobra_model)

### Engine List
| Engine                       | IO                      | Output Model             |
|------------------------------|-------------------------|--------------------------|
| MetabolicSBML                | SBML file               | MewpyMetabolic           |
| BooleanRegulatoryCSV         | CSV file                | RegulatoryModel          |
| CobraModelEngine             | cobra.core.model        | CobraWrapper             |
| ReframedModelEngine          | reframed.core.cbmodel   | ReframedWrapper          |
| CoExpressionRegulatoryCSV    | CSV file                | RegulatoryModel          |
| TargetRegulatorRegulatoryCSV | CSV file                | RegulatoryModel          |
| JSON                         | JSON File               | MewpyMetabolic           |
| RegulatorySBML               | SBML file               | RegulatoryModel          |


## Exemplos de ler GERM models

#### 1. Read a Cobrapy model instance

In [16]:

cobra_model = read_sbml_model("models/e_coli_core.xml")
cobra_reader = Reader(Engines.CobraModelEngine, cobra_model)

read_model(cobra_reader) # output: mewpy.germ.models.cobra_wrapper.CobraModelWrapper

Model,e_coli_core
Name,E. coli core model - Orth et al 2010
Types,metabolic
Compartments,"c, e"
Reactions,95
Metabolites,72
Genes,137
Exchanges,20
Demands,0
Sinks,0
Objective,Biomass_Ecoli_core


#### 2. Read a regulatory model from a CSV file

In [15]:
trn_reader = Reader(Engines.BooleanRegulatoryCSV, "models/e_coli_core_trn.csv")

read_model(trn_reader) # output: mewpy.germ.models.regulatory.RegulatoryModel

Model,e_coli_core_trn
Name,model
Types,regulatory
Compartments,
Regulatory interactions,160
Targets,160
Regulators,160
Regulatory reactions,0
Regulatory metabolites,0
Environmental stimuli,76


#### 3. Read a regulatory model from a CSV and a metabolic model from a SMBL

In [18]:
trn_reader = Reader(Engines.BooleanRegulatoryCSV, "models/e_coli_core_trn.csv")
gem_reader = Reader(Engines.MetabolicSBML, "models/e_coli_core.xml")

read_model(trn_reader, gem_reader) # output: mewpy.germ.models.model.RegulatoryMewpy_MetabolicModel

Model,e_coli_core
Name,E. coli core model - Orth et al 2010
Types,"regulatory, metabolic"
Compartments,"c, e"
Reactions,95
Metabolites,72
Genes,137
Exchanges,20
Demands,0
Sinks,0
Objective,Biomass_Ecoli_core


#### GERM/analysis
##### Module para permitir correr simulações e análises aos GERM models

In [19]:
!tree ../src/mewpy/germ/analysis -L 1 -I "__*"

../src/mewpy/germ/analysis
├── analysis_utils.py
├── coregflux.py
├── fba.py
├── integrated_analysis.py
├── metabolic_analysis.py
├── pfba.py
├── prom.py
├── regulatory_analysis.py
├── rfba.py
└── srfba.py

0 directories, 10 files


| File                    | Methods                                      | Scope                |
|-------------------------|----------------------------------------------|----------------------|
| fba.py                  | FBA                                          | Metabolic            |
| pfba.py                 | pFBA                                         | Metabolic            |
| metabolic_analysis.py   | FVA, single gene deletion, single reaction deletion  | Metabolic            |
| rfba.py                 | rFBA                                         | Regulatory-Metabolic |
| srfba.py                | SR-FBA                                       | Regulatory-Metabolic |
| prom.py                 | PROM                                         | Regulatory-Metabolic |
| coregflux.py            | CoRegFlux                                    | Regulatory-Metabolic |
| integrated_analysis.py  | iFVA, integrated single gene deletion, integrated single reaction deletion  | Regulatory-Metabolic |
| regulatory_analysis.py  | Regulatory truth table                       | Regulatory           |


#### Metabolic Methods behaviour
<img src="images/met_method_flowchart.png" alt="Metabolic Methods behaviour" width="600">

#### Metabolic methods examples

In [20]:
cobra_model = read_sbml_model("models/e_coli_core.xml")
cobra_reader = Reader(Engines.CobraModelEngine, cobra_model)

met_model_cobra = read_model(cobra_reader) # mewpy.germ.models.cobra_wrapper.CobraModelWrapper

In [21]:
gem_reader = Reader(Engines.MetabolicSBML, "models/e_coli_core.xml")

met_model_mewpy = read_model(gem_reader) # mewpy.germ.models.model.Mewpy_MetabolicModel

In [22]:
from mewpy.germ.analysis.fba import FBA

In [23]:
# Como o cobrapy implementa o método metabólico FBA, Vai ser essa a implementação usada

FBA(met_model_cobra).build().optimize()

Method,FBA
Model,Model e_coli_core - E. coli core model - Orth et al 2010
Objective,Biomass_Ecoli_core
Objective value,0.8739215069684302
Status,optimal


In [24]:
# Como os modelos mewpy_metabolic não têm simulator, o método has_external_method dá sempre falso,
# por isso para resolver um modelo metbólico como o FBA, è utilizado o solver do mewpy

FBA(met_model_mewpy).build().optimize()

Method,FBA
Model,Model e_coli_core - E. coli core model - Orth et al 2010
Objective,Biomass_Ecoli_core
Objective value,0.8739215069684303
Status,optimal
